#### Введение
На лекции обсуждалось полезное свойство VCG аукциона, что каждому участнику выгодно в качестве ставки использовать свою частную ценность товара `value`, а менять её в зависимости от ставок других игроков - не выгодно.

Ваша цель - реализовать аукцион с таким свойством.

#### Модель рекламного объявления
В данной задаче, каждое объявление имеет свою частную ценность `value`, а также (тут отличие от примера с лекции) свою собственную вероятноть `prob` быть кликнутым в случае если пользователь дойдет до позиции на которой показывается объявление.

По умолчанию стратегия ставки у каждого объявления - ставить свою ценность `value` (см. `get_bid` у `Advert`)

Однако, мы хотим исследовать будет ли выгодно ставить значение отличное от `value`. Рекламное объявление, с иной стратегией ставки описывается классом `DeviantAdvert`.

In [ ]:
class Advert(object):
    def init(self, gen):
        self.prob = gen.random()
        self._value = gen.random()*9+1
        return self
    
    def __repr__(self):
        return 'Prob: %0.3f, Value: %0.3f' % (
            self.prob, self._value
        )
    
    def get_bid(self, ads):
        return self._value
    
    
class DeviantAdvert(Advert):
    def __init__(self, get_bid_func):
        self.get_bid_func = get_bid_func
        
    def get_bid(self, ads):
        if self.get_bid_func is not None:
            bid = self.get_bid_func(self, ads)
            assert bid >= 0.0
            return bid
        return Advert.get_bid(self, ads)

#### Задача

Далее, в качестве примера, описано три варианта аукциона. 

Функция аукциона принимает на вход список объектов класса `Advert`, а также список априорных вероятностей клика на каждую из рекламных позиций участвующих в аукционе. 
Можно считать, что вероятность клика на объявление `a`, поставленное на позицию `i`, будет равна `a.prob * position_prob[i]`

Каждый из аукционов в качестве победителей выбирает объявления с максимальным матожиданием денег - `a.prob * a.get_bid()`

Разница между ними в цене за клик.
Аукцион "первой цены" в качестве цены за клик предлагает собственно свою ставку.
Аукцион "следующей цены" в качестве цены за клик предлагает ставку следующего.

Вам предлагается реализовать рассчет цены клика для аукциона vcg.

In [ ]:
def auction_first_price(ads, position_prob):
    n = len(position_prob)
    winners = sorted(ads, key=lambda ad: ad.prob * ad.get_bid(ads), reverse=True)[:n]
    costs = [ad.get_bid(ads) for ad in winners]
    return winners, costs

def auction_next_price(ads, position_prob):
    n = len(position_prob)
    winners = sorted(ads, key=lambda ad: ad.prob * ad.get_bid(ads), reverse=True)[:n + 1]
    costs = [winners[i + 1].get_bid(ads) for i in range(len(winners) - 1)]
    del winners[-1]
    return winners, costs


def auction_vcg(ads, position_prob):
    #
    # кодить сюда
    # лекция, страницы 37+, https://yadi.sk/i/v_ifDDKFDqb8tQ
    # для рассчетов нельза использовать ad._value, так как он нам какбе неизвестен
    #    

#### Тест №1
Для базового тестирования предлагается пример рассмотренный на лекции. В нем частные вероятности объявлений отсутствуют, поэтому мы положим их равными единице. 
Кликабельности позиций из лекции равные 100 и 80 превратим в вероятности 1.0 и 0.8
Правильно реализованный vcg должен пройти этот тест

In [ ]:
def test_from_lecture(auc, answer):
    ads = [Advert() for i in range(3)]
    ads[0]._value = 10
    ads[1]._value = 4
    ads[2]._value = 2
    for i in range(3):
        ads[i].prob = 1
        
    result = auc(ads, [1.0, 0.8])[1]
    
    error = max(abs(ri - ai) for ri, ai in zip(result, answer))
    if error > 1e-7:
        print(result, '!=', answer)
    else:
        print(auc.__name__, 'is fine!')
    
test_from_lecture(auction_first_price, [10, 4])
test_from_lecture(auction_next_price, [4, 2])
test_from_lecture(auction_vcg, [2.4, 2.0])

#### Тест №2+

Далее идет много кода суть которого сводится к следующему.

Сгенерируем случайное множество рекламных объявлений.
Все объявления кроме одного будут придерживаться "честной" стратегии ставки и ставить свою ценность `value`. 
Одно объявление будет реализовывать иную стратегию, задаваемую параметром `deviant_advert_func`.

Далее разыграем аукцион и оценим профит полученный "девиантом". Сравним профит с вариантом, когда "девиант" поступает честно. Если профит от честной стратегии меньше, чем от нечестной - то тест аукциона считается проваленным.

Правильно написанный аукцион vcg должен проходить такого рода тест для любой возможной deviant_advert_func. То есть никакая стратегия ставки не должна быть лучше, чем честная.

В режиме debug=True выводится информация о первом проваленом аукционе: кандидаты, ставки, рассчитанная стоимость.

In [ ]:
from tqdm import tqdm
from random import Random

def test_deviant_strategy(auc, position_prob, deviant_advert_func=None, seed=42, debug=False):
    assert position_prob == sorted(position_prob, reverse=True)
    
    doc_count = len(position_prob) + 1    
    gen = Random(seed)    
    for it in range(10000):
        # генерируем объявления
        ads = [Advert().init(gen) for i in range(doc_count - 1)]
        ads.append(DeviantAdvert(deviant_advert_func).init(gen))
        deviant = ads[-1]

        # запускаем аукцион
        winners, costs = auc(ads, position_prob)
        assert len(winners) == len(position_prob), 'Something goes wrong'
        assert min(costs) > 0, 'No free lunch'
        
        cur_deviant_profit = 0.0
        for p, (ad, cost) in zip(position_prob, zip(winners, costs)):
            if ad == deviant:
                # матожидание профита равно вероятности клика
                # умноженной на ценность минус стоимость клика
                cur_deviant_profit = (ad._value - cost) * ad.prob * p
                break
        
        if deviant_advert_func is not None:
            # разыгрываем аукцион как будто девиант ведет себя честно
            deviant.get_bid_func = None
            winners_bl, costs_bl = auc(ads, position_prob)
            
            honest_deviant_profit = 0.0
            for p, (ad, cost) in zip(position_prob, zip(winners_bl, costs_bl)):
                if ad == deviant:
                    honest_deviant_profit = (ad._value - cost) * ad.prob * p
                    break
                    
            # если честный профит меньше нечестного - фейлим тест
            if cur_deviant_profit > honest_deviant_profit:
                if debug:
                    print('Fail')
                    print('Deviant auction results:')
                    for ad in ads:
                        try:
                            idx = winners.index(ad)
                            print('Ad:', ad, 'Win:', idx, 'Bid:', deviant_advert_func(ad, ads), 'Cost:', costs[idx])
                        except:
                            print('Ad:', ad)
                    print()
                    print('Honest auction results:')
                    for ad in ads:
                        try:
                            idx = winners_bl.index(ad)
                            print('Ad:', ad, 'Win:', idx, 'Bid:', Advert.get_bid(ad, ads), 'Cost:', costs[idx])
                        except:
                            print('Ad:', ad)
                    print()
                    print()
                    
                return False, cur_deviant_profit - honest_deviant_profit
            
    return True, 0.0

#### Стратегии ставок

Далее для примера приводится несколько стратегий ставок отличных от честной.

Первая стратегия: ставить `value - eps` для некоторого параметра `eps`.

Вторая стратегия: ставить такое значение, чтобы переместиться на `pos_delta` позиций ниже по аукциону и еще плюс `eps`.

Вы вольны предложить свои стратегии ставок, которые вы считаете уменее, чем предложенные.

In [ ]:
def create_eps_strategy(eps):
    def minus_eps_bid(self, ads):
        return max(self._value - eps, 1e-5)
    return minus_eps_bid

def create_eps_pos_strategy(pos_delta, eps):
    def minus_eps_pos_bid(self, ads):
        winners = sorted(ads, key=lambda ad: ad.prob * ad._value, reverse=True)
        idx = winners.index(self)
        idx = max(0, min(idx + pos_delta, len(ads) - 1))
        return max(winners[idx]._value * winners[idx].prob / self.prob + eps, 1e-5)
    return minus_eps_pos_bid

STRATEGIES = [
    create_eps_strategy(+0.5),
    create_eps_strategy(-0.5),
    create_eps_strategy(+2),
    create_eps_strategy(-2),
    create_eps_pos_strategy(+1, +1e-5),
    create_eps_pos_strategy(-1, +1e-5),
    create_eps_pos_strategy(+1, -1e-5),
    create_eps_pos_strategy(-1, -1e-5),
    create_eps_pos_strategy(+5, +1e-5),
    create_eps_pos_strategy(-5, +1e-5),
    create_eps_pos_strategy(+5, -1e-5),
    create_eps_pos_strategy(-5, -1e-5),
]

def test_deviant_strategies(aucs):
    # пример вероятностей клика по позиции
    # можно менять на своё усмотрение    
    position_prob = [0.9, 0.7, 0.5, 0.4]
    for auc in aucs:
        for si, strategy in enumerate(STRATEGIES):
            for n in range(1, len(position_prob) + 1):
                ok, profit = test_deviant_strategy(auc, position_prob[:n], deviant_advert_func=strategy, debug=False)
                if not ok:
                    auc_name = auc.__name__
                    strategy_name = strategy.__name__ 
                    print('Auction `%s` is cheated with strategy `%d: %s` with profit %s' % (auc_name, si, strategy_name, profit))                    
    print('Done')

#### Тестируем простые аукционы
Предложенные аукционы первой и следующей цены проваливают тест на половине стратегий

In [ ]:
test_deviant_strategies([auction_first_price, auction_next_price])

#### Тестируем Ваш код
Правильно написанный аукцион vcg должен пройти все тесты

In [ ]:
test_deviant_strategies([auction_vcg])